In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb 

p1_model = xgb.XGBClassifier()
p2_model = xgb.XGBClassifier()
p1_model.load_model('p1_method_target_xgboost_model.json')
p2_model.load_model('p2_method_target_xgboost_model.json')
cleaned_df = pd.read_csv('ufc_cleaned.csv')
fighters_df = pd.read_csv('ufc_fighters_cleaned.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
cleaned_df.head()

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED,p1_height,p1_weight,p1_reach,p1_stance,p1_dob,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_dob,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.
0,Cory Sandhagen,Deiveson Figueiredo,0,0,0.62,0.66,1.00,0.40,0,0,1,1,307,31,0,0,0.79,1.00,1.00,0.25,0,0,0,0,217,4,0.62,0.66,0.79,1.00,KO/TKO,Dan Miragliotta,1,2025-05-03,54,87,6,9,92,149,8,11,2,2,2,5,31,39,2,2,53,78,3,3,1,1,1,4,33,65,4,6,2,2,0,0,19,20,2,3,24,43,4,7,0,0,2,2,30,44,0,0,22,30,1,1,1,1,0,0,8,8,1,1,10,13,1,1,0,0,1,1,21,26,0,0,71.0,135.0,70.0,Switch,1992-04-20,5.02,0.44,3.46,0.56,1.19,0.32,0.63,0.3,65.0,135.0,68.0,Orthodox,1987-12-18,2.91,0.54,3.55,0.49,1.62,0.34,0.60,1.4
1,Reinier de Ridder,Bo Nickal,1,0,0.68,0.38,0.00,1.00,0,0,0,0,129,144,0,0,0.50,0.33,0.00,1.00,0,0,0,0,112,120,0.68,0.38,0.50,0.33,KO/TKO,Mike Beltran,1,2025-05-03,22,32,8,21,91,104,24,40,0,3,1,1,5,10,2,6,43,49,17,24,0,2,1,1,9,18,2,13,13,14,4,5,0,0,2,3,7,15,3,14,15,17,5,7,0,0,0,0,2,7,1,5,3,3,1,1,0,0,0,0,1,5,0,4,4,5,2,2,0,0,0,0,76.0,185.0,78.0,Southpaw,1990-09-07,3.28,0.57,2.69,0.38,6.89,0.46,1.00,2.0,73.0,185.0,76.0,Southpaw,1996-01-14,2.65,0.63,1.86,0.64,3.61,0.50,0.00,3.6
2,Santiago Ponzinibbio,Daniel Rodriguez,0,1,0.44,0.39,0.12,NaN,0,0,0,0,44,7,0,0,0.42,0.43,0.00,NaN,0,0,0,0,2,7,0.44,0.39,0.42,0.43,KO/TKO,Kevin MacDonald,0,2025-05-03,49,111,66,166,50,115,70,170,1,8,0,0,15,35,35,81,15,35,35,81,0,2,0,0,31,83,39,131,6,16,20,25,12,12,7,10,49,111,64,164,0,0,1,1,0,0,1,1,5,19,20,61,3,9,13,16,7,7,2,4,15,35,34,80,0,0,1,1,0,0,0,0,72.0,170.0,73.0,Orthodox,1986-09-26,4.80,0.41,4.61,0.61,0.56,0.32,0.71,0.0,73.0,170.0,74.0,Southpaw,1986-12-31,7.50,0.49,5.47,0.58,0.59,0.54,0.57,0.1
3,Montel Jackson,Daniel Marcos,0,0,0.45,0.53,0.25,0.42,1,0,0,0,217,345,0,0,0.33,0.57,0.25,1.00,0,0,0,0,127,86,0.45,0.53,0.33,0.57,Decision - Unanimous,Gary Copeland,1,2025-05-03,30,66,17,32,67,106,24,43,2,8,3,7,4,12,4,7,22,31,7,11,1,4,1,1,25,58,5,18,1,3,5,7,4,5,7,7,27,62,13,27,3,4,3,4,0,0,1,1,2,8,2,4,1,2,1,2,1,2,1,1,3,11,3,6,1,1,1,1,0,0,0,0,70.0,135.0,75.0,Southpaw,1992-04-24,3.39,0.53,1.42,0.63,3.41,0.71,0.69,0.3,67.0,135.0,69.0,Orthodo

In [4]:
def getData(p1, p2, eventDate, ref): # Takes Two fighters, event date, and referee, and creates the features again for the model
    cols = ['height','weight','reach','SLpM','Str. Acc.', 'SApM','Str. Def','TD Avg.','TD Acc.','TD Def.','Sub. Avg.']
    
    row1 = fighters_df[fighters_df['name'] == p1]
    row2 = fighters_df[fighters_df['name'] == p2]

    if row1.empty or row2.empty:
        missing = p1 if row1.empty else p2
        raise ValueError(f"Fighter not found: {missing}")

    f1 = row1[cols].squeeze().to_numpy(dtype=float)
    f2 = row2[cols].squeeze().to_numpy(dtype=float)
    fighter_diff = f1 - f2


    #calculate age at event
    eventDate = pd.to_datetime(eventDate)
    p1Dob = pd.to_datetime(fighters_df.loc[fighters_df['name'] == p1, 'dob'].iloc[0])
    p2Dob = pd.to_datetime(fighters_df.loc[fighters_df['name'] == p2, 'dob'].iloc[0])
    p1Age = (eventDate - p1Dob).days / 365.25
    p2Age = (eventDate - p2Dob).days / 365.25
    ageDiff = p1Age - p2Age


    #calculate age adjusted stats
    age_adjust_cols = ['SLpM','Str. Acc.', 'SApM','Str. Def','TD Avg.','TD Acc.','TD Def.','Sub. Avg.']
    p1_age_adjusted = {}
    p2_age_adjusted = {}
    
    for col in age_adjust_cols:
        if col in row1.columns and col in row2.columns:
            p1_value = row1[col].squeeze()
            p2_value = row2[col].squeeze()
            
            # Apply age adjustment formula
            p1_age_adjusted[col] = p1_value * (1/p1Age)
            p2_age_adjusted[col] = p2_value * (1/p2Age)



    #find days since last fight
    p1_fights = cleaned_df[(cleaned_df['p1_fighter'] == p1) | (cleaned_df['p2_fighter'] == p1)]
    p2_fights = cleaned_df[(cleaned_df['p1_fighter'] == p2) | (cleaned_df['p2_fighter'] == p2)]
    
    p1_last = pd.to_datetime(p1_fights['event_date']).max() if not p1_fights.empty else pd.NaT
    p2_last = pd.to_datetime(p2_fights['event_date']).max() if not p2_fights.empty else pd.NaT
    
    p1_days = (eventDate - p1_last).days if pd.notnull(p1_last) else None
    p2_days = (eventDate - p2_last).days if pd.notnull(p2_last) else None
    days_diff = p1_days - p2_days


    #get stance and encode it
    stance1 = row1['stance'].squeeze()
    stance2 = row2['stance'].squeeze()
    categories = ['Open Stance', 'Orthodox', 'Sideways', 'Southpaw', 'Switch']
    stance1 = [True if stance1 == cat else False for cat in categories]
    stance2 = [True if stance2 == cat else False for cat in categories]


    #calculate records
    p1_win_fights = cleaned_df[((cleaned_df['p1_fighter'] == p1) & (cleaned_df['winner'] == 1)) | ((cleaned_df['p2_fighter'] == p1) & (cleaned_df['winner'] == 0))]
    p1_loss_fights = cleaned_df[((cleaned_df['p1_fighter'] == p1) & (cleaned_df['winner'] == 0)) | ((cleaned_df['p2_fighter'] == p1) & (cleaned_df['winner'] == 1))]

    p1_wins = len(p1_win_fights)
    p1_losses = len(p1_loss_fights)
    p1_total = p1_wins + p1_losses
    
    p2_win_fights = cleaned_df[((cleaned_df['p1_fighter'] == p2) & (cleaned_df['winner'] == 0)) | ((cleaned_df['p2_fighter'] == p2) & (cleaned_df['winner'] == 1))]
    p2_loss_fights = cleaned_df[((cleaned_df['p1_fighter'] == p2) & (cleaned_df['winner'] == 1)) | ((cleaned_df['p2_fighter'] == p2) & (cleaned_df['winner'] == 0))]

    p2_wins = len(p2_win_fights)
    p2_losses = len(p2_loss_fights)
    p2_total = p2_wins + p2_losses
    
    # Calculate differences
    win_diff = p1_wins - p2_wins
    loss_diff = p1_losses - p2_losses
    total_diff = p1_total - p2_total


    # calculate win streaks
    p1_past_fights = cleaned_df[
        (cleaned_df['event_date'].notna()) & 
        ((cleaned_df['p1_fighter'] == p1) | (cleaned_df['p2_fighter'] == p1))
    ].sort_values('event_date', ascending=False)
    
    p2_past_fights = cleaned_df[
        (cleaned_df['event_date'].notna()) & 
        ((cleaned_df['p1_fighter'] == p2) | (cleaned_df['p2_fighter'] == p2))
    ].sort_values('event_date', ascending=False)
    
    # Calculate p1's current win streak
    p1_win_streak = 0
    for _, fight in p1_past_fights.iterrows():
        # Check if p1 won this fight
        if (fight['p1_fighter'] == p1 and fight['winner'] == 1) or (fight['p2_fighter'] == p1 and fight['winner'] == 0):
            p1_win_streak += 1
        else:
            # Stop counting at first loss
            break
    
    # Calculate p2's current win streak
    p2_win_streak = 0
    for _, fight in p2_past_fights.iterrows():
        # Check if p2 won this fight
        if (fight['p1_fighter'] == p2 and fight['winner'] == 1) or (fight['p2_fighter'] == p2 and fight['winner'] == 0):
            p2_win_streak += 1
        else:
            # Stop counting at first loss
            break
    

    #referee
    ref_counts = (cleaned_df['referee'].value_counts()).get(ref, 0)


    #calculate EMA
    features = [
    'KD', 'SIG_STR_PCT', 'TD_PCT', 'SUB_ATT', 'REV', 'CTRL',
    'R1_KD', 'R1_SIG_STR_PCT', 'R1_TD_PCT', 'R1_SUB_ATT', 'R1_REV', 'R1_CTRL',
    'SIG_STR_PCT_DETAILED', 'R1_SIG_STR_PCT_DETAILED',
    'SIG_STR_LANDED', 'SIG_STR_ATTEMPTED', 'TOTAL_STR_LANDED', 'TOTAL_STR_ATTEMPTED',
    'TD_LANDED', 'TD_ATTEMPTED',
    'R1_SIG_STR_LANDED', 'R1_SIG_STR_ATTEMPTED', 'R1_TOTAL_STR_LANDED', 'R1_TOTAL_STR_ATTEMPTED',
    'R1_TD_LANDED', 'R1_TD_ATTEMPTED',
    'HEAD_LANDED', 'HEAD_ATTEMPTED', 'BODY_LANDED', 'BODY_ATTEMPTED',
    'LEG_LANDED', 'LEG_ATTEMPTED',
    'DISTANCE_LANDED', 'DISTANCE_ATTEMPTED', 'CLINCH_LANDED', 'CLINCH_ATTEMPTED',
    'GROUND_LANDED', 'GROUND_ATTEMPTED',
    'R1_HEAD_LANDED', 'R1_HEAD_ATTEMPTED', 'R1_BODY_LANDED', 'R1_BODY_ATTEMPTED',
    'R1_LEG_LANDED', 'R1_LEG_ATTEMPTED',
    'R1_DISTANCE_LANDED', 'R1_DISTANCE_ATTEMPTED', 'R1_CLINCH_LANDED', 'R1_CLINCH_ATTEMPTED',
    'R1_GROUND_LANDED', 'R1_GROUND_ATTEMPTED'
    ]   
    
    # Initialize dictionaries to store EMAs
    p1_emas = {}
    p2_emas = {}
    
    # Calculate EMAs for p1
    cleaned_df['event_date'] = pd.to_datetime(cleaned_df['event_date'])
    p1_prev_fights = cleaned_df[(cleaned_df['event_date'] < eventDate) & ((cleaned_df['p1_fighter'] == p1) | (cleaned_df['p2_fighter'] == p1))]
    p1_prev_fights = p1_prev_fights.sort_values('event_date', ascending=False).head(3)
    
    if not p1_prev_fights.empty:
        # Process each fight for p1
        for _, fight in p1_prev_fights.iterrows():
            position = 'p1' if fight['p1_fighter'] == p1 else 'p2'
            prefix = 'p1_' if position == 'p1' else 'p2_'
            
            # Extract feature values for this fight
            for feat in features:
                col_name = f"{prefix}{feat}"
                if col_name in fight and not pd.isna(fight[col_name]):
                    # Convert to numeric and add to list
                    if feat not in p1_emas:
                        p1_emas[feat] = []
                    
                    p1_emas[feat].append(pd.to_numeric(fight[col_name], errors='coerce'))

        for feat in features:
            if feat in p1_emas:
                if len(p1_emas[feat]) == 1:
                    # Only one fight, use that value
                    p1_emas[feat] = p1_emas[feat][0]
                elif len(p1_emas[feat]) == 2:
                    # Two fights, weight more recent fight higher (0.6, 0.4)
                    p1_emas[feat] = 0.6 * p1_emas[feat][0] + 0.4 * p1_emas[feat][1]
                elif len(p1_emas[feat]) >= 3:
                    # Three or more fights, use weights 0.5, 0.3, 0.2 for most recent to oldest
                    p1_emas[feat] = 0.5 * p1_emas[feat][0] + 0.3 * p1_emas[feat][1] + 0.2 * p1_emas[feat][2]
            else:
                p1_emas[feat] = np.nan
    
    # Calculate EMAs for p2
    p2_prev_fights = cleaned_df[(cleaned_df['event_date'] < eventDate) & 
                               ((cleaned_df['p1_fighter'] == p2) | (cleaned_df['p2_fighter'] == p2))]
    p2_prev_fights = p2_prev_fights.sort_values('event_date', ascending=False).head(3)
    
    if not p2_prev_fights.empty:
        # Process each fight for p2
        for _, fight in p2_prev_fights.iterrows():
            position = 'p1' if fight['p1_fighter'] == p2 else 'p2'
            prefix = 'p1_' if position == 'p1' else 'p2_'
            
            # Extract feature values for this fight
            for feat in features:
                col_name = f"{prefix}{feat}"
                if col_name in fight and not pd.isna(fight[col_name]):
                    # Convert to numeric and add to list
                    if feat not in p2_emas:
                        p2_emas[feat] = []
                    
                    p2_emas[feat].append(pd.to_numeric(fight[col_name], errors='coerce'))
        
        # Calculate simple weighted average for the last 3 fights (most recent has highest weight)
        for feat in features:
            if feat in p2_emas:
                if len(p2_emas[feat]) == 1:
                    # Only one fight, use that value
                    p2_emas[feat] = p2_emas[feat][0]
                elif len(p2_emas[feat]) == 2:
                    # Two fights, weight more recent fight higher (0.6, 0.4)
                    p2_emas[feat] = 0.6 * p2_emas[feat][0] + 0.4 * p2_emas[feat][1]
                elif len(p2_emas[feat]) >= 3:
                    # Three or more fights, use weights 0.5, 0.3, 0.2 for most recent to oldest
                    p2_emas[feat] = 0.5 * p2_emas[feat][0] + 0.3 * p2_emas[feat][1] + 0.2 * p2_emas[feat][2]
            else:
                p2_emas[feat] = np.nan


        # List of grouped methods to track
        temp_df = cleaned_df.copy()
        
        # --- Method Grouping Section ---
        # Create method_grouped column FIRST
        temp_df['method_grouped'] = temp_df['method'].replace({
            'Decision - Majority': 'Decision',
            'Decision - Split': 'Decision',
            'Decision - Unanimous': 'Decision',
            "TKO - Doctor's Stoppage": "KO/TKO",
            **{cat: 'Other' for cat in ['Overturned', 'Could Not Continue', 'DQ', 'Other']}
        })
        
        # --- Win/Loss Calculations Using method_grouped ---
        # Calculate p1 wins with method grouping
        p1_win_fights = temp_df[
            ((temp_df['p1_fighter'] == p1) & (temp_df['winner'] == 1)) |
            ((temp_df['p2_fighter'] == p1) & (temp_df['winner'] == 0))
        ]
        
        # Calculate p2 wins with method grouping
        p2_win_fights = temp_df[
            ((temp_df['p1_fighter'] == p2) & (temp_df['winner'] == 0)) |
            ((temp_df['p2_fighter'] == p2) & (temp_df['winner'] == 1))
        ]
        
        # --- Method-Specific Win Counts ---
        method_categories = ['Decision', 'KO/TKO', 'Submission']
        
        # Fighter 1 method wins
        p1_method_wins = {
            method: len(p1_win_fights[p1_win_fights['method_grouped'] == method])
            for method in method_categories
        }
        
        # Fighter 2 method wins
        p2_method_wins = {
            method: len(p2_win_fights[p2_win_fights['method_grouped'] == method])
            for method in method_categories
        }
       
        
        feature_dict = {
        'winner': np.nan,  # Placeholder for prediction
        
        # Basic stats for fighter 1
        'p1_height': f1[0],
        'p1_weight': f1[1],
        'p1_reach': f1[2],
        'p1_slpm': f1[3],
        'p1_str_acc': f1[4],
        'p1_sapm': f1[5],
        'p1_str_def': f1[6],
        'p1_td_avg': f1[7],
        'p1_td_acc': f1[8],
        'p1_td_def': f1[9],
        'p1_sub_avg': f1[10],
        
        # Basic stats for fighter 2
        'p2_height': f2[0],
        'p2_weight': f2[1],
        'p2_reach': f2[2],
        'p2_slpm': f2[3],
        'p2_str_acc': f2[4],
        'p2_sapm': f2[5],
        'p2_str_def': f2[6],
        'p2_td_avg': f2[7],
        'p2_td_acc': f2[8],
        'p2_td_def': f2[9],
        'p2_sub_avg': f2[10],
        
        # Age and physical differences
        'p1_age_at_event': p1Age,
        'p2_age_at_event': p2Age,
        'height_diff': f1[0] - f2[0],
        'reach_diff': f1[1] - f2[1],
        'weight_diff': f1[2] - f2[2],
        'age_diff': ageDiff,
        
        # Skill differences
        'slpm_diff': f1[3] - f2[3],
        'stracc_diff': f1[4] - f2[4],
        'sapm_diff': f1[5] - f2[5],
        'strdef_diff': f1[6] - f2[6],
        'tdavg_diff': f1[7] - f2[7],
        'tdacc_diff': f1[8] - f2[8],
        'tddef_diff': f1[9] - f2[9],
        'subavg_diff': f1[10] - f2[10],
        
        # Time since last fight
        'p1_days_since_last_fight': p1_days,
        'p2_days_since_last_fight': p2_days,
        'days_since_last_fight_diff': days_diff,
        
        # Fight records
        'p1_wins': p1_wins,
        'p1_losses': p1_losses,
        'p1_total': p1_total,
        'p2_wins': p2_wins,
        'p2_losses': p2_losses,
        'p2_total': p2_total,
        'win_diff': win_diff,
        'loss_diff': loss_diff,
        'total_diff': total_diff,
        'p1_win_streak': p1_win_streak,
        'p2_win_streak': p2_win_streak,
        
        # Age adjusted stats
        'p1_age_adjusted_slpm': p1_age_adjusted['SLpM'],
        'p2_age_adjusted_slpm': p2_age_adjusted['SLpM'],
        'p1_age_adjusted_str_acc': p1_age_adjusted['Str. Acc.'],
        'p2_age_adjusted_str_acc': p2_age_adjusted['Str. Acc.'],
        'p1_age_adjusted_sapm': p1_age_adjusted['SApM'],
        'p2_age_adjusted_sapm': p2_age_adjusted['SApM'],
        'p1_age_adjusted_str_def': p1_age_adjusted['Str. Def'],
        'p2_age_adjusted_str_def': p2_age_adjusted['Str. Def'],
        'p1_age_adjusted_td_avg': p1_age_adjusted['TD Avg.'],
        'p2_age_adjusted_td_avg': p2_age_adjusted['TD Avg.'],
        'p1_age_adjusted_td_acc': p1_age_adjusted['TD Acc.'],
        'p2_age_adjusted_td_acc': p2_age_adjusted['TD Acc.'],
        'p1_age_adjusted_td_def': p1_age_adjusted['TD Def.'],
        'p2_age_adjusted_td_def': p2_age_adjusted['TD Def.'],
        'p1_age_adjusted_sub_avg': p1_age_adjusted['Sub. Avg.'],
        'p2_age_adjusted_sub_avg': p2_age_adjusted['Sub. Avg.']
    }
    
    # Add all EMA metrics for both fighters
    for feat in features:
        feature_dict[f'p1_{feat.lower()}_ema'] = p1_emas[feat]
        feature_dict[f'p2_{feat.lower()}_ema'] = p2_emas[feat]
    

    # Method-specific wins
    feature_dict.update({
    'p1_decision_wins': p1_method_wins['Decision'],
    'p1_ko/tko_wins': p1_method_wins['KO/TKO'],
    'p1_submission_wins': p1_method_wins['Submission'],
    
    'p2_decision_wins': p2_method_wins['Decision'],
    'p2_ko/tko_wins': p2_method_wins['KO/TKO'],
    'p2_submission_wins': p2_method_wins['Submission'],
    })
    
    # Add stance encoding
    for i, stance_cat in enumerate(categories):
        feature_dict[f'p1_stance_{stance_cat}'] = stance1[i]
    for i, stance_cat in enumerate(categories):
        feature_dict[f'p2_stance_{stance_cat}'] = stance2[i]
    
    # Add referee frequency
    feature_dict['referee_freq'] = ref_counts

    # Convert to DataFrame
    prediction_df = pd.DataFrame([feature_dict])
    
    return prediction_df


def predict_winner(p1, p2, eventDate, ref):
    fight_features = getData(p1, p2, eventDate, ref)
    fight_features_numeric = fight_features.drop(columns=['winner']).astype(float)

    def validate_features(input_features, target_model):
        # Load the required features for this model
        required_features = pd.read_json(f"{target_model}_features.json", typ='series').tolist()
        
        # Check for missing features
        missing = [f for f in required_features if f not in input_features]
        if missing:
            raise ValueError(f"Missing required features: {missing}")
        
        # Return only the required features in correct order
        return input_features[required_features]

    p1_features = validate_features(fight_features, 'p1_method_target')
    p2_features = validate_features(fight_features, 'p2_method_target')
    
    p1_probs = p1_model.predict_proba(p1_features)
    p2_probs = p2_model.predict_proba(p2_features)
    
    print(p1_probs) # (Desicion, KO/TKO, Submission)

    return p1_probs, p2_probs

def convert_to_american_odds(prob):
    if prob == 0:
        return float('inf')  # Or some large number if zero prob
    elif prob >= 0.5:
        return int(round(- (prob / (1 - prob)) * 100))
    else:
        return int(round(((1 - prob) / prob) * 100))

def calculate_fight_odds(p1_probs, p2_probs):
    class_names = ['Decision', 'KO/TKO', 'Submission']  # Default order
    
    p1_probs = p1_probs.flatten()
    p2_probs = p2_probs.flatten()
    
    odds = {}
    for i, method in enumerate(class_names):
        p1_odd = convert_to_american_odds(p1_probs[i])
        p2_odd = convert_to_american_odds(p2_probs[i])
        odds[method] = {
            'p1': f"{'+' if p1_odd > 0 else ''}{p1_odd}",
            'p2': f"{'+' if p2_odd > 0 else ''}{p2_odd}"
        }
    print(odds)
    return odds

p1_probs, p2_probs = predict_winner('Belal Muhammad', 'Jack Della Maddalena', '2025-05-10', 'Jason Herzog')
fight_odds = calculate_fight_odds(p1_probs, p2_probs)


[[0.6453875  0.34518653 0.00942606]]
{'Decision': {'p1': '-182', 'p2': '-155'}, 'KO/TKO': {'p1': '+190', 'p2': '+398'}, 'Submission': {'p1': '+10509', 'p2': '+422'}}
